In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import torchvision.transforms.v2 as transforms
import matplotlib.pyplot as plt
from ResidualAutoEncoder import ResidualAutoencoder, wandb_sweep, visualize_images, denormalize
from ResidualAutoEncoder import Criterion
import time



c:\Users\vinme\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\vinme\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you

In [ ]:
import wandb
wandb.login()


In [ ]:
sweep_config = {
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'num_blocks': {
            'values': [2, 3, 4]
        },
        'block_depth': {
            'values': [2, 3]
        },
        'bottleneck_dim': {
            'values': [64, 128, 256]
        },
        'asym_block': {
            'values': [1]
        },
        'asym_depth': {
            'values': [1]
        },
        'learning_rate': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-6,
        'max': 0.01,
        'min': 1e-6,
        },
        'l1_lambda': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-15,
        'max': 1e-6,
        'min': 1e-15,
        },
        'l1_lambda_bottleneck': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-10,
        'max': 1e-4,
        'min': 1e-10,
        },
        'lambda_kl': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-10,
        'max': 1e-4,
        'min': 1e-10,
        },
        'lambda_tv': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-19,
        'max': 1e-8,
        'min': 1e-19,
        },
        'lambda_perceptual': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-6,
        'max': 1e-1,
        'min': 1e-6,
        },
        'lambda_mse': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e1,
        'min': 1e1,
        'max': 1e2,
        },
        'lambda_ssim': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-6,
        'max': 1e-1,
        'min': 1e-6,
        },
        'lambda_psnr': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-19,
        'max': 1e-8,
        'min': 1e-19,
        },
        'dropout': {
            'values': [0.2, 0.3, 0.4]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="residual-autoencoder")


In [ ]:
wandb.agent(sweep_id, wandb_sweep, count=25)

In [2]:
transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
        ])

dataset = datasets.ImageFolder(root='data/water-meter/', transform=transform)
#dataset.targets = dataset.samples

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=6, pin_memory=True)
test_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=6, pin_memory=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = ResidualAutoencoder(num_blocks=4, block_depth=2, bottleneck_dim=256, channels=3, device=device)

criterion = Criterion(model=model)

optimizer = optim.Adam(model.parameters(), lr=0.000001)

model.train_harness(model, train_loader, test_loader, criterion, optimizer, epochs=10, wandb_log=False)

AttributeError: module 'torchvision.transforms.v2' has no attribute 'ToImage'

In [ ]:
for data in train_loader:
    inputs, outputs = data
    plt.imshow(inputs[0].permute(1, 2, 0))
    plt.imshow(outputs[0].permute(1, 2, 0))
    break

In [ ]:
def unnormalize(tensor):    
    return tensor * 0.5 + 0.5

In [5]:

# Hyperparameters
batch_size = 16
learning_rate = 0.000001
epochs = 10

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformation and loading
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
    
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=3, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)



# Model, loss function, and optimizer
model = ResidualAutoencoder(num_blocks=4, block_depth=2, bottleneck_dim=256, channels=3, dropout=0.1, device=device)
criterion = Criterion(model=model)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
model.train_harness(model=model, train_loader= train_loader, criterion=criterion, optimizer=optimizer, epochs= epochs, test_loader=test_loader)

#torch.save(model.state_dict(), 'models/residual_autoencoder.pth')
# Evaluate the model
inputs, outputs, val = model.evaluate_harness(model, test_loader, device)
outputs = denormalize(outputs)

visualize_images(inputs)
visualize_images(outputs)
# Visualize the results
# num_images = 10
# fig, axes = plt.subplots(2, num_images, figsize=(15, 4))
# for i in range(num_images):
#     axes[0, i].imshow(inputs[i].permute(1, 2, 0))
#     axes[0, i].axis('off')
#     axes[1, i].imshow(outputs[i].permute(1, 2, 0))
#     axes[1, i].axis('off')

# plt.show()
    
def unnormalize(tensor):
    return tensor * 0.5 + 0.5



Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 42.6452, recon_loss: 0.2396, val_loss: 0.2318 learning rate: 1e-06


KeyboardInterrupt: 

In [ ]:
dataset = datasets.ImageFolder(root='data/water-meter/', transform=transform)
#dataset.targets = dataset.samples

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=6, pin_memory=True)
test_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=6, pin_memory=True)

In [ ]:
# Train the model
model.train_harness(model=model, train_loader= train_loader, criterion=criterion, optimizer=optimizer, epochs=epochs, test_loader=test_loader)

#torch.save(model.state_dict(), 'models/residual_autoencoder.pth')
# Evaluate the model
inputs, outputs, val = model.evaluate_harness(model, test_loader, device)
outputs = unnormalize(outputs)
# Visualize the results
num_images = 10
fig, axes = plt.subplots(2, num_images, figsize=(15, 4))
for i in range(num_images):
    axes[0, i].imshow(inputs[i].permute(1, 2, 0))
    axes[0, i].axis('off')
    axes[1, i].imshow(outputs[i].permute(1, 2, 0))
    axes[1, i].axis('off')

plt.show()

In [ ]:
script_model = model.optimize_for_inference(type='lite', pruning_amount=0.1)
script_model = torch.jit.optimize_for_inference(script_model)

curr_time = time.time()
inputs, outputs, val = model.evaluate_harness(model, test_loader, torch.device('cpu'))
outputs = unnormalize(outputs)
# Visualize the results
num_images = 10
fig, axes = plt.subplots(2, num_images, figsize=(15, 4))
for i in range(num_images):
    axes[0, i].imshow(inputs[i].permute(1, 2, 0))
    axes[0, i].axis('off')
    axes[1, i].imshow(outputs[i].permute(1, 2, 0))
    axes[1, i].axis('off')

plt.show()
print(val)
final_time = time.time()
print(final_time - curr_time)

curr_time = time.time()
inputs, outputs, val = model.evaluate_harness(script_model, test_loader, torch.device('cpu'))
outputs = unnormalize(outputs)
# Visualize the results
num_images = 10
fig, axes = plt.subplots(2, num_images, figsize=(15, 4))
for i in range(num_images):
    axes[0, i].imshow(inputs[i].permute(1, 2, 0))
    axes[0, i].axis('off')
    axes[1, i].imshow(outputs[i].permute(1, 2, 0))
    axes[1, i].axis('off')

plt.show()
print(val)
final_time = time.time()
print(final_time - curr_time)



In [ ]:
print(outputs[0].shape)
#plt.imshow(transforms.ToPILImage()(outputs[0].cpu()))